https://habr.com/ru/post/564916/

In [1]:
from pathlib import Path
from custom_types import Sample
from locate_spans import locate_spans
from custom_datasets import IterableJsonDataset
from tqdm import tqdm

In [2]:
data_dir = Path("./data/news_org/")

In [3]:
import torch
from transformers import FSMTModel, FSMTTokenizer, FSMTForConditionalGeneration

In [4]:
tokenizer = FSMTTokenizer.from_pretrained("facebook/wmt19-en-ru")
model = FSMTForConditionalGeneration.from_pretrained("facebook/wmt19-en-ru")

inverse_tokenizer = FSMTTokenizer.from_pretrained("facebook/wmt19-ru-en")
inverse_model = FSMTForConditionalGeneration.from_pretrained("facebook/wmt19-ru-en")

In [1]:
model.cuda();
inverse_model.cuda();

In [1]:
def paraphrase(samples: list[Sample], gram=4, num_beams=5, **kwargs):
    """ Generate a paraphrase using back translation. 
    Parameter `gram` denotes size of token n-grams of the original sentence that cannot appear in the paraphrase.
    """
    # Encoding
    texts = [s.text for s in samples]
    input_ids = inverse_tokenizer.batch_encode(texts, return_tensors="pt")
    
    with torch.no_grad():
        outputs = inverse_model.generate(input_ids.to(inverse_model.device), num_beams=num_beams, **kwargs)
    # Translated
    other_lang = inverse_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    # print(other_lang)
    
    # Bad ids calculation
    input_ids = input_ids[0, :-1].tolist()
    bad_word_ids = [input_ids[i:(i+gram)] for i in range(len(input_ids)-gram)]
    
    # Backtranslation
    input_ids = tokenizer.encode(other_lang, return_tensors="pt")
    with torch.no_grad():
        # Constrained back translation shall be in a loop
        # bc cannot use force ids with batches
        outputs = model.generate(input_ids.to(model.device), num_beams=num_beams, bad_words_ids=bad_word_ids, **kwargs)
    
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

In [5]:
dataset = IterableJsonDataset(data_dir)

In [6]:
total = 0
for _ in IterableJsonDataset(data_dir):
    total+=1
total

136171

In [40]:
from torch.nn.functional import pad

In [64]:
pad(output, (0, 250 - output.shape[1]), value=tokenizer.pad_token_id).shape

torch.Size([1, 250])

In [72]:
data_iterator = iter(dataset)
batch_size = 8
num_beams=5
gram=4
for i in tqdm(range(total)):
    batch = []

    # Not all entities are good
    while len(batch) != batch_size:
        sample = next(data_iterator)
        if len(sample.ent) < 3:
            continue
        if len(inverse_tokenizer(sample.text, truncation=False)["input_ids"]) > 200:
            continue

        batch.append(sample)
    # ---------------------
    samples = batch
    
    # Encoding
    texts = [s.text for s in samples]
    encoded_input = inverse_tokenizer(texts, return_tensors="pt", max_length=200, padding=True, truncation=True)
    decoded_input = [tokenizer.batch_decode(x) for x in encoded_input["input_ids"]]
    # Align tokens with entity span
    entity_spans = locate_spans(decoded_input, samples)
    
    
    with torch.no_grad():
        inputs = encoded_input["input_ids"].to(inverse_model.device)
        
        outputs = inverse_model.generate(inputs=inputs, num_beams=num_beams, max_new_tokens=250)
    
    
    # Translated
    other_lang = inverse_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    # print(other_lang)
    

    # Good ids calculation
    force_words_ids = []
    for inp, span in zip(inputs, entity_spans):
        entity_tokens = inp[span[0]:span[1]+1].tolist()
        force_words_ids.append(entity_tokens)
    
    # Bad ids calculation
    bad_word_ids = []
    for text, good_ids in zip(texts, force_words_ids):
        input_ids = inverse_tokenizer.encode(text)
        candidates = [input_ids[i:(i+gram)] for i in range(len(input_ids)-gram)]
        # TODO check intersection with force_words_ids
        bad_word_ids.append(candidates)

    
    # Backtranslation
    input_ids = tokenizer(other_lang, return_tensors="pt", max_length=250, padding=True, truncation=True)
    with torch.no_grad():
        # Constrained back translation shall be in a loop
        # bc cannot use force ids with batches
        inputs = input_ids.to(model.device)
        outputs = []
        for inp, atm, force_words_id, bad_word_id in zip(inputs["input_ids"], inputs["attention_mask"], force_words_ids, bad_word_ids):
            inp = inp.unsqueeze(0)
            output = model.generate(
                inputs=inp,
                num_beams=num_beams, 
                bad_words_ids=bad_word_id,
                force_words_ids=[force_words_id],
                repetition_penalty=3.14,
            )
            output = pad(output, (0, 250 - output.shape[1]), value=tokenizer.pad_token_id)
            outputs.append(output)
    
    outputs = torch.concat(outputs)
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    # return decoded
    break
    # ---------------------

    

  0%|                                                                                                                                 | 0/136171 [00:00<?, ?it/s]/home/max/code/research_mention_decoder/.venv/lib/python3.10/site-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|                                                                                                                                 | 0/136171 [01:29<?, ?it/s]


In [75]:
for sp, d in zip(samples, decoded):
    print(sp.text)
    print(d)
    print("-----------")

На прошлой неделе министр энергетики Белоруссии Виктор Каранкевич сообщил, что Минск намерен возобновить газовые переговоры с Москвой до конца января.О ходе переговоров Минска и Москвы на поставку сырья в 2020 году — в материале “Ъ” «Нефти контракты не писаны».О резком сокращении поставок российского газа — в материале “Ъ” «"Газпром" снижает экспорт в Европу».
О ходе переговоров между Минском и Москвой по поставкам сырья на 2020 год - в материале Ъ "Нефтяные контракты не пишутся", о резком снижении поставок газа из России - в материале Ъ "Газпром" наращивает экспорт в Европу.
-----------
Белоруссия закупила 80 тыс. тонн норвежской нефти, сообщил представитель госконцерна «Белнефтехим».
"Беларус" закупил 80 тыс. тонн нефти, сообщил журналистам представитель госконБелнефтехима. В настоящее время компания закупает около 20 тыс. тонн нефти в год. По его словам, объем поставок составляет примерно $100 млн. Это на 30% больше, чем в прошлом году. На сегодняшний день поставки составляют более 

In [59]:
for o in outputs:
    print(o.shape)
output.shape

torch.Size([1, 33])


torch.Size([1, 33])

In [37]:
torch.concat(outputs).shape

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 50 but got size 34 for tensor number 1 in the list.

In [36]:
tokenizer.batch_decode(outputs[0], skip_special_tokens=True)

['Глава международного отдела Hyundai Йоланда Лэй отмечает, что компания отправила антисептики и инсектициды пострадавшим в Ханчжоу, но она ожидает, что государственные власти Китая держат ситуацию на контроле.']

In [25]:
for sample, bwi in zip(samples, bad_word_ids):
    print(sample.text)
    for bw in bwi:
        print(tokenizer.decode(bw))

Пока это только пилотный проект, его тестируют в закрытом режиме на покупателях магазина «Азбука вкуса» в деловом центре «Москва-Сити».
Пока это только пило
это только пилотный
только пилотный проект
пилотный проект,
тный проект, его
проект, его тести
, его тестируют
его тестируют в
тестируют в закры
руют в закрытом
в закрытом режиме
закрытом режиме на
том режиме на покупа
режиме на покупателях
на покупателях магазина
покупателях магазина "
телях магазина "А
магазина "Аз
"Азбу
Азбука
збука вку
бука вкуса
ка вкуса "
вкуса "в
са "в дело
"в деловом
в деловом центре
деловом центре "
вом центре "Москва
центре "Москва-
"Москва-Сити
Москва-Сити "
-Сити ".
Возможность отказаться от писем есть и сейчас: сервис «Почты России» zakaznoe.pochta.ru позволяет гражданам при желании получать юридически значимые документы исключительно в электронном виде.
Возможность отказаться от
ность отказаться от писем
отказаться от писем есть
от писем есть и
писем есть и сейчас
есть и сейчас:
и сейчас: сервис
сейча

In [17]:
entity_spans

[(16, 20), (11, 12), (12, 16), (26, 27), (1, 3), (30, 32), (17, 17), (4, 7)]

In [18]:
samples

[Sample(text='Подобные расследования идут по всей Европе, самым громким за последние годы стал скандал вокруг Danske Bank — по мнению следствия, через его эстонское подразделение могли отмывать около €200 млрд.Яна Рождественская', span=(96, 107), ent='Danske Bank'),
 Sample(text='В прошлом году другой местный финансовый конгломерат, ING, согласился выплатить €775 млн за прекращение дела об отмывании.', span=(54, 57), ent='ING'),
 Sample(text='Власти Нидерландов начали расследование в отношении одного из крупнейших местных банков, ABN Amro, его подозревают в причастности к отмыванию денег.', span=(89, 97), ent='ABN Amro'),
 Sample(text='Конкурсный управляющий полагает, что деньги были выведены из компании.Подробнее — в материале “Ъ” «Кредиторы "Вия" добрались до Китая».', span=(110, 113), ent='Вия'),
 Sample(text='«Автокомплект» утверждал, что предоставил компании целевой заем на 30, 5 млн руб. на производство фильма «Вий 3D», который не был возвращен.', span=(1, 13), ent='Автокомплект'

In [35]:
texts

['В ноябре 2018 года суд признал компанию банкротом и открыл конкурсное производство.На сайте Palmali сообщается, что она основана в 2000 году.',
 'Есть немного продуктов, которые приготовить можем только на костре", - цитируются также в сообщении слова капитана танкера Вадима Каючкина.Группа компаний Palmali, согласно информации на ее сайте, основана в Турции в 1998 году.',
 'На все требования моряков вернуть их домой Palmali отвечает молчанием."По сути, компания просто бросила судно вместе с экипажем, никакой реакции ни на наши обращения, ни на сообщения моряков нет.',
 'Сейчас прорабатываем технические вопросы", - сказал он.Трубопровод "Роснефти" Оха - Комсомольск-на-Амуре был перекрыт в июле после нефтеразлива близ Хабаровска на площади 0, 04 га.',
 'Росприроднадзор оценил ущерб от происшествия в Норильске в рекордные 148, 2 млрд рублей.',
 'Сейчас можно было переносить 100%, законопроект предлагает не более 50% на период 2021-2023 гг.Поправки к законопроекту принимаются профильным